In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
#instantiating llms

from langchain_groq import ChatGroq
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings

embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
rag_llm = ChatGroq(model="llama-3.3-70b-versatile")

/tmp/ipykernel_129847/4069931954.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [4]:
#parsing the pdf
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader

#setting up parser
parser = LlamaParse(
    result_type = "markdown"
)

file_extractor = {".pdf":parser}
documents = SimpleDirectoryReader(input_files = ['FOOD MENU 20-01-25 TO 26-01-25.pdf'], file_extractor=file_extractor).load_data()

Started parsing the file under job_id 9f2a5856-da23-4d41-8ffa-963ef775a63e


In [5]:
print(documents)

[Document(id_='e9f7105c-acdf-4107-ad09-e632c23837e0', embedding=None, metadata={'file_path': 'FOOD MENU 20-01-25 TO 26-01-25.pdf', 'file_name': 'FOOD MENU 20-01-25 TO 26-01-25.pdf', 'file_type': 'application/pdf', 'file_size': 188106, 'creation_date': '2025-01-22', 'last_modified_date': '2025-01-22'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='# Mahindra University Mess Menu From 20-01-25 To 26-01-25\n\n|Days|Breakfast|Lunch|Dinner|Snacks|\n|---|---|---|---|---|\n|MONDAY|PLAIN AND PALAK WADA|VEG SALAD(CARROT+CUCUMBER)|RUMALI ROTI|CORN SAMOSA|\n| |UGGANI|GHEE ROTI|KADAI PANEER|BOILED PEANUTS|\n| |CHOCOLATE MUFFINS|BROWN ONION PULAO|STE

In [6]:
menu_items = documents[0].text
menu_items

'# Mahindra University Mess Menu From 20-01-25 To 26-01-25\n\n|Days|Breakfast|Lunch|Dinner|Snacks|\n|---|---|---|---|---|\n|MONDAY|PLAIN AND PALAK WADA|VEG SALAD(CARROT+CUCUMBER)|RUMALI ROTI|CORN SAMOSA|\n| |UGGANI|GHEE ROTI|KADAI PANEER|BOILED PEANUTS|\n| |CHOCOLATE MUFFINS|BROWN ONION PULAO|STEAMED RICE|TEA|\n| |TEA|EGG CHETTINAD|JEERA RICE|COFFEE|\n| |COFFEE|SOYA AND BEANS CURRY|TOMATO RASAM|BOOST|\n| |BOOST|STEAMED RICE|DAL MAKHANI| |\n| |MILK|TOMATO DAL|CURD , PICKLE| |\n| |CORN FLAKES|CORIANDER RASAM|CUT FRUITS| |\n| |GINGER CHUTNEY|TOMATO PACHADI| | |\n| |PEANUT CHUTNEY|GULAB JAMUN| | |\n| |SAMBAR| | | |\n|TUESDAY|ONION UTHAPPAM|VEG SALAD|VEG MANCHOW SOUP|ONION PAKODI|\n| |PEANUT POHA|POORI|CHICKEN MANCHOW SOUP|SPROUT SALAD|\n| |CHOCOCHIP COOKIES|CHOLE MASALA (KABULI CHANA)|WET MANCHURIAN|TEA|\n| |TEA|BEETROOT PORIYAL|SCHEZWAN EGG FRIED RICE|BOOST|\n| |COFFEE|JEERA RICE|SCHEZWAN VEG FRIED RICE|COFFEE|\n| |BOOST|STEAMED RICE|VEG NOODLES| |\n| |MILK|THOTAKURA DAL|EGG NOODLES| |\n|

In [7]:
# Change stream=True to stream=False
from groq import Groq

client = Groq()
completion = client.chat.completions.create(
    messages=[
      # Set an optional system message. This sets the behavior of the
        # assistant and can be used to provide specific instructions for
        # how it should behave throughout the conversation.
        {
            "role": "system",
            "content": "You are an AI assistant with the job to summarise the given markdown menu, When you see the day in the first column of the given table, start a fresh day with the headings as breakfast lunch dinner snacks."
        },
        # Set a user message for the assistant to respond to.
        {
            "role": "user",
            "content": menu_items,
        }
    ],   # Your messages
    model="llama-3.3-70b-versatile",
    stream=False  # ← Changed to False
)

# Access full response directly
complete_response = completion.choices[0].message.content
print(complete_response)

## Monday
### Breakfast
- Plain and palak wada
- Uggani
- Chocolate muffins
- Tea
- Coffee
- Boost
- Milk
- Corn flakes
- Ginger chutney
- Peanut chutney
- Sambar

### Lunch
- Veg salad (carrot+cucumber)
- Ghee roti
- Brown onion pulao
- Egg chettinad
- Soya and beans curry
- Steamed rice
- Tomato dal
- Coriander rasam
- Tomato pachadi
- Gulab jamun

### Dinner
- Rumali roti
- Kadai paneer
- Steamed rice
- Jeera rice
- Tomato rasam
- Dal makhani
- Cut fruits

### Snacks
- Corn samosa
- Boiled peanuts
- Tea
- Coffee
- Boost

## Tuesday
### Breakfast
- Onion uthappam
- Peanut poha
- Chocochip cookies
- Tea
- Coffee
- Boost
- Milk
- Chocos, boiled eggs
- Tomato chutney

### Lunch
- Veg salad
- Poori
- Chole masala (kabuli chana)
- Beetroot poriyal
- Jeera rice
- Steamed rice
- Thotakura dal
- Tomato rasam
- Beerakaya pachadi

### Dinner
- Veg manchow soup
- Chicken manchow soup
- Schezwán egg fried rice
- Schezwán veg fried rice
- Veg noodles
- Egg noodles
- Steamed rice
- Yellow dal tadk

In [8]:
menu_summary = complete_response

In [9]:
menu_summary

'## Monday\n### Breakfast\n- Plain and palak wada\n- Uggani\n- Chocolate muffins\n- Tea\n- Coffee\n- Boost\n- Milk\n- Corn flakes\n- Ginger chutney\n- Peanut chutney\n- Sambar\n\n### Lunch\n- Veg salad (carrot+cucumber)\n- Ghee roti\n- Brown onion pulao\n- Egg chettinad\n- Soya and beans curry\n- Steamed rice\n- Tomato dal\n- Coriander rasam\n- Tomato pachadi\n- Gulab jamun\n\n### Dinner\n- Rumali roti\n- Kadai paneer\n- Steamed rice\n- Jeera rice\n- Tomato rasam\n- Dal makhani\n- Cut fruits\n\n### Snacks\n- Corn samosa\n- Boiled peanuts\n- Tea\n- Coffee\n- Boost\n\n## Tuesday\n### Breakfast\n- Onion uthappam\n- Peanut poha\n- Chocochip cookies\n- Tea\n- Coffee\n- Boost\n- Milk\n- Chocos, boiled eggs\n- Tomato chutney\n\n### Lunch\n- Veg salad\n- Poori\n- Chole masala (kabuli chana)\n- Beetroot poriyal\n- Jeera rice\n- Steamed rice\n- Thotakura dal\n- Tomato rasam\n- Beerakaya pachadi\n\n### Dinner\n- Veg manchow soup\n- Chicken manchow soup\n- Schezwán egg fried rice\n- Schezwán veg f

In [10]:
print(menu_summary)

## Monday
### Breakfast
- Plain and palak wada
- Uggani
- Chocolate muffins
- Tea
- Coffee
- Boost
- Milk
- Corn flakes
- Ginger chutney
- Peanut chutney
- Sambar

### Lunch
- Veg salad (carrot+cucumber)
- Ghee roti
- Brown onion pulao
- Egg chettinad
- Soya and beans curry
- Steamed rice
- Tomato dal
- Coriander rasam
- Tomato pachadi
- Gulab jamun

### Dinner
- Rumali roti
- Kadai paneer
- Steamed rice
- Jeera rice
- Tomato rasam
- Dal makhani
- Cut fruits

### Snacks
- Corn samosa
- Boiled peanuts
- Tea
- Coffee
- Boost

## Tuesday
### Breakfast
- Onion uthappam
- Peanut poha
- Chocochip cookies
- Tea
- Coffee
- Boost
- Milk
- Chocos, boiled eggs
- Tomato chutney

### Lunch
- Veg salad
- Poori
- Chole masala (kabuli chana)
- Beetroot poriyal
- Jeera rice
- Steamed rice
- Thotakura dal
- Tomato rasam
- Beerakaya pachadi

### Dinner
- Veg manchow soup
- Chicken manchow soup
- Schezwán egg fried rice
- Schezwán veg fried rice
- Veg noodles
- Egg noodles
- Steamed rice
- Yellow dal tadk

In [11]:
#set up vector database

from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.embeddings import HuggingFaceEmbeddings


text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n## ", "\n### "],  # Split on the meal type headers
    chunk_size=1000,  # Adjust this value to control the chunk size
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)

# Split the text into chunks
chunks = text_splitter.split_text(menu_summary)

# Convert the chunks into Document objects
documents = [Document(page_content=chunk) for chunk in chunks]

vectorstore = Chroma.from_documents(documents, embedding = embed_model ,collection_name="embedded_menu")
retriever=vectorstore.as_retriever()

print(f"Documents indexed: {len(documents)}")

Documents indexed: 5


In [12]:
documents

[Document(metadata={}, page_content='## Monday\n### Breakfast\n- Plain and palak wada\n- Uggani\n- Chocolate muffins\n- Tea\n- Coffee\n- Boost\n- Milk\n- Corn flakes\n- Ginger chutney\n- Peanut chutney\n- Sambar\n\n### Lunch\n- Veg salad (carrot+cucumber)\n- Ghee roti\n- Brown onion pulao\n- Egg chettinad\n- Soya and beans curry\n- Steamed rice\n- Tomato dal\n- Coriander rasam\n- Tomato pachadi\n- Gulab jamun\n\n### Dinner\n- Rumali roti\n- Kadai paneer\n- Steamed rice\n- Jeera rice\n- Tomato rasam\n- Dal makhani\n- Cut fruits\n\n### Snacks\n- Corn samosa\n- Boiled peanuts\n- Tea\n- Coffee\n- Boost'),
 Document(metadata={}, page_content='## Tuesday\n### Breakfast\n- Onion uthappam\n- Peanut poha\n- Chocochip cookies\n- Tea\n- Coffee\n- Boost\n- Milk\n- Chocos, boiled eggs\n- Tomato chutney\n\n### Lunch\n- Veg salad\n- Poori\n- Chole masala (kabuli chana)\n- Beetroot poriyal\n- Jeera rice\n- Steamed rice\n- Thotakura dal\n- Tomato rasam\n- Beerakaya pachadi\n\n### Dinner\n- Veg manchow 

In [13]:
await retriever.ainvoke("Whats for lunch on wednesday")

[Document(id='50c3819c-2034-4f18-bfeb-b6f4035b6e02', metadata={}, page_content='## Wednesday\n### Breakfast\n- Plain/ragi idly\n- Semiya bath\n- Fruit cake\n- Tea\n- Coffee\n- Boost\n- Milk\n- Chocos\n- Ginger chutney\n- Sambar\n\n### Lunch\n- Onion and cucumber salad\n- Rumali roti\n- Hyderabadi chicken curry\n- Panneer butter masala (thick gravy)\n- Bagara rice\n- Steamed rice\n- Palak dal\n- Pepper rasam\n- Gongura and onion pachadi\n\n### Dinner\n- Masala uthapam\n- Dal kichidi\n- Tomato pachadi\n- Peanut chutney\n- Steamed rice\n- Gongura dal\n- Curd\n- Cut fruits\n- Kadu ka kheer\n\n### Snacks\n- Maggie\n- Chudwa\n- Tea\n- Coffee\n- Boost\n- Maggie masala (Nestle)'),
 Document(id='7064bc8b-6791-4552-a865-fd48bb6647ed', metadata={}, page_content='## Monday\n### Breakfast\n- Plain and palak wada\n- Uggani\n- Chocolate muffins\n- Tea\n- Coffee\n- Boost\n- Milk\n- Corn flakes\n- Ginger chutney\n- Peanut chutney\n- Sambar\n\n### Lunch\n- Veg salad (carrot+cucumber)\n- Ghee roti\n- Bro

In [25]:
#setting up RAG pipeline

from langchain_core.documents import Document
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables.passthrough import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from typing import List,Dict

RAG_SYSTEM_PROMPT = """\
You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context given within delimiters to answer the human's questions. You can also use your own knowledge to answer these questions. 
```
{context}
```
If you don't know the answer, just say that you don't know.\
"""

RAG_HUMAN_PROMPT = "{input}"

RAG_PROMPT = ChatPromptTemplate.from_messages([
    ("system", RAG_SYSTEM_PROMPT),
    ("human", RAG_HUMAN_PROMPT)
])

def format_docs(docs: List [Document]):
    """Format the Retrieved Documents"""
    return "\n".join(doc.page_content for doc in docs)

rag_chain = (
    {
        "context": retriever|format_docs,
        "input": RunnablePassthrough()
    }
    |RAG_PROMPT
    |rag_llm
    |StrOutputParser()
)

In [28]:
user_input = input("Enter your Query:\n")
response = await rag_chain.ainvoke(user_input)
print(" ")
print(response)

Enter your Query:
 What can I eat on wednesday, Im trying to cut carbs


 
On Wednesday, it seems like there are a few options that might be lower in carbs. For breakfast, you could try the fruit cake, but be aware that it's likely to contain some sugar and carbs. Alternatively, you could opt for the milk, tea, or coffee.

For lunch, the onion and cucumber salad might be a good choice, as it's likely to be low in carbs. You could also try the Hyderabadi chicken curry, as it's a protein-rich dish that might be served without too many carbs. However, be mindful of the rumali roti and steamed rice, as they are high in carbs.

For dinner, the peanut chutney and curd might be good options, as they are relatively low in carbs. You could also try the gongura dal, but be aware that it might contain some carbs from the dal itself.

For snacks, the tea, coffee, and milk are all low in carbs. The chudwa might be a bit higher in carbs, but it depends on the ingredients used to make it.

Keep in mind that it's always a good idea to consult with a healthcare professional